In [ ]:
import requests
from bs4 import BeautifulSoup
from user_agent import generate_user_agent

In [ ]:
def default_bs4_headers():
    headers = {
        "accept": "text/html,application/xhtml+xml,application/xml,application/json;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
        "accept-language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7,ja;q=0.6",
        "sec-ch-ua": '" Not A;Brand";v="99", "Chromium";v="90", "Google Chrome";v="90"',
        "sec-ch-ua-mobile": "?0",
        "sec-fetch-dest": "document",
        "sec-fetch-mode": "navigate",
        "sec-fetch-site": "none",
        "sec-fetch-user": "?1",
        "upgrade-insecure-requests": "1",
        "cache-control": "max-age=0",
        "accept-encoding": "gzip, deflate, br",
        "user-agent": generate_user_agent(),
    }
    return headers

In [ ]:
default_url = """https://gichulpass.com/bbs/board.php?bo_table=exam&wr_id={}"""

In [ ]:
default_url.format(1)

In [ ]:
num_dic = {'①':1, '②':2, '③':3, '④':4, '⑤':5}

In [ ]:
# url = default_url.format(page_no)
# print(url)
res = requests.get("https://gichulpass.com/bbs/board.php?bo_table=exam&wr_id=815", headers=default_bs4_headers())
soup = BeautifulSoup(res.content, "html.parser")
title = soup.select_one(".wr_title")

# title = title.text.strip()
# # print(title)
examList = soup.select("#examList > li")
cho_lst = []
for exam in examList:
    question = exam.select_one('.pr_problem').text.strip()
    example = exam.select_one('.example2')
    if example:
        print(example.decode_contents(formatter="html").strip())
        break
    choices = exam.select('.questionCon > li > label')
    for cho in choices:
        cho_ = cho.decode_contents(formatter="html").strip()
        cho_lst.append(cho_)
    # print(cho_lst)
    answer_explan = exam.select_one('.answer_explan').text.strip()
    answer_explan = answer_explan
    # print(answer_explan)
    answer_num = exam.select_one('.answer_num').text.strip()
    answer_num = num_dic[answer_num]
    # print(answer_num)

In [ ]:
result = []
for page_no in range(1405):
    url = default_url.format(page_no)
    print(url)
    res = requests.get(url, headers=default_bs4_headers())
    soup = BeautifulSoup(res.content, "html.parser")
    title = soup.select_one(".wr_title")
    if not title:
        continue
    title = title.text.strip()
    title_lst = title.split(' ')
    year = title_lst[0]
    subject = ' '.join(title_lst[-3:])
    print(year, subject)
    examList = soup.select("#examList > li")
    for exam in examList:
        try:
            question = exam.select_one('.pr_problem').text.strip()
            # print(question)
            choices = exam.select('.questionCon > li > label')
            paragraph = exam.select_one('.example2')
            if paragraph:
                paragraph = paragraph.decode_contents(formatter="html").strip()    
            cho_lst = []
            for cho in choices:
                cho_ = cho.decode_contents(formatter="html").strip()
                cho_lst.append(cho_)
            # print(cho_lst)
            answer_explan = exam.select_one('.answer_explan').text.strip()
            answer_explan = answer_explan
            # print(answer_explan)
            answer_num = exam.select_one('.answer_num').text.strip()
            answer_num = num_dic[answer_num]
            # print(answer_num)
            result.append([year, subject, paragraph, question, cho_lst, answer_explan, answer_num, url])
        except:
            print('error: ', url)

In [ ]:
# 수능형 문제

# 수능의 국어, 사회 영역(윤리, 정치, 사회)과 비슷한 문제

# KMMLU (Korean History), MMMLU (HighSchool 데이터 중 역사, 경제, 정치, 지리, 심리)

# KLUE MRC (경제, 교육산업, 국제, 부동산, 사회, 생활, 책마을)

In [1]:
import pandas as pd

In [ ]:
df = pd.DataFrame(result, columns=['year', 'subject', 'paragraph', 'question', 'cho_lst', 'answer_explan', 'answer', 'url'])

In [ ]:
df.to_csv('open_exam_all.csv', encoding='utf-8-sig')

In [4]:
df = pd.read_csv('open_exam_all.csv', index_col = 0)

In [9]:
# selected_subject = ['국어', '사회', '한국사', '경제학', '교육학']
selected_subject = ['국어', '사회', '한국사', '사회복지학']

final = []
for i, r in df.iterrows():
    r_subject = r[1].split('급')[-1].strip().split(' ')[0].strip()
    if r_subject in selected_subject:
        r[1] = r_subject
        final.append(r)

/tmp/ipykernel_289914/3063806427.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  r_subject = r[1].split('급')[-1].strip().split(' ')[0].strip()
/tmp/ipykernel_289914/3063806427.py:8: FutureWarning: Series.__setitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To set a value by position, use `ser.iloc[pos] = value`
  r[1] = r_subject


In [10]:
df_final = pd.DataFrame(final, columns=['year', 'subject', 'paragraph', 'question', 'cho_lst', 'answer_explan', 'answer', 'url'])

In [11]:
df_final

,year,subject,paragraph,question,cho_lst,answer_explan,answer,url
0,2008년,국어,NaN,1. 밑줄 친 부분의 표기가 옳은 것은?,['&nbsp;①&nbsp;지영은 수험생인 아들에게 한약 한 <u>재</u>를 먹였...,②목욕재계(沐浴齋戒 : 부정(不淨)을 타지 않도록 깨끗이 목욕하고 몸가짐을 가다듬는...,2,https://gichulpass.com/bbs/board.php?bo_table=...
1,2008년,국어,NaN,2. 국어 로마자 표기법에 맞는 단어들로만 묶인 것은?,"['&nbsp;①&nbsp;서울 Seoul, 부산 Busan, 대전 Daejeon,...","①광주 Gwangju\r\n②놓다 nota, 좋고 joko\r\n④팔당 Paltan...",3,https://gichulpass.com/bbs/board.php?bo_table=...
2,2008년,국어,NaN,3. 한자 성어에서 같은 음절이 겹쳐지면 대체로 강조의 의미를 갖는다. 강조의 의미...,"['&nbsp;①&nbsp;疊疊山中', '&nbsp;②&nbsp;戀戀不忘', '&n...",한자 성어에서 같은 음절이 겹쳐지면 대체로 강조의 의미를 갖는다.\r\n① 疊疊山中...,4,https://gichulpass.com/bbs/board.php?bo_table=...
3,2008년,국어,NaN,4. 밑줄 친 부분이 맞춤법에 맞는 것은?,['&nbsp;①&nbsp;정희하고 수철이는 약속 시간을 지켜서 <u>왔을껄</u>...,③에서 어미 ‘-ㄹ는지’가 바른 표기이다.\r\n①왔을걸.(o)\r\n②‘~학생이데...,3,https://gichulpass.com/bbs/board.php?bo_table=...
4,2008년,국어,ㄱ.언문 창제는 사대모화(事大慕華)의 도리에 어긋난다.<br/>\r\nㄴ.한자 아닌...,5. 다음은 훈민정음 창제를 반대한 최만리의 상소 내용을 요약한 것이다. 반대의 논...,"['&nbsp;①&nbsp;ㄱ', '&nbsp;②&nbsp;ㄴ', '&nbsp;③&...","ㄱ, ㄴ, ㄷ은 사대모화(事大慕華) 사상에 입각한 한자 중심의 사고(언어적 측면)와...",4,https://gichulpass.com/bbs/board.php?bo_table=...
...,...,...,...,...,...,...,...,...
24783,2022년(2회),한국사,"ㄱ. 조선을 중화로, 청을 오랑캐로 보는 명분론으로 이어진다. <br/>\r\nㄴ....",16. <보기>의 조선 후기 호락논쟁에 대한 설명 중 성격이 다른 것은?,"['&nbsp;①&nbsp;ㄱ', '&nbsp;②&nbsp;ㄴ', '&nbsp;③&...",※호락논쟁\r\n 노론 내부의 심성론 논쟁으로 18세기 중엽 노론 내부에 주기설과 ...,2,https://gichulpass.com/bbs/board.php?bo_table=...
24784,2022년(2회),한국사,오늘 우리는 전 세계 이목이 우리를 주시하는 가운데 40년 독재정치를 청산하고 희망...,17. <보기>의 선언문이 발표된 이후에 일어난 변 화로 가장 옳은 것은?,"['&nbsp;①&nbsp;해방 이후 단절되었던 일본과의 국교가 정상화 되었다', ...",※1987년 6월 10일 발표된 국민대회선언문\n\r\n④ [○] 1987년 1월 ...,4,https://gichulpass.com/bbs/board.php?bo_table=...
24785,2022년(2회),한국사,"재위 12년 신미년에 왕이 거칠부 및 대각찬 구진, 각찬 비태, 잡찬 탐지, 잡찬 ...",18. <보기>의 밑줄 친 ‘왕’의 재위 기간에 일어난 일이 아닌 것은?,"['&nbsp;①&nbsp;대가야를 정벌하여 가야 연맹을 소멸시켰다', '&nbsp...","※신라의 진흥왕 - 거칠부 / 대각찬,대아찬(골품과 관등)\n\r\n③ [Ⅹ] 황룡...",3,https://gichulpass.com/bbs/board.php?bo_table=...
24786,2022년(2회),한국사,"ㄱ. 음식 조리과정에서 왜간장, 조미료 등을 사용 하였다. <br/>\r\nㄴ. 도...",19. <보기>에서 일제 강점기의 의식주 변화에 해당하는 것을 모두 고른 것은?,"['&nbsp;①&nbsp;ㄱ,ㄷ', '&nbsp;②&nbsp;ㄱ,ㄷ', '&nbs...","ㄱ[○]음식 조리과정에서 왜간장, 조미료 등을 사용하였다. \r\nㄴ[○] 도시 외...",4,https://gichulpass.com/bbs/board.php?bo_table=...


In [12]:
df_final.to_csv('open_exam_select2.csv', encoding='utf-8-sig')